Cartar de posible desvio para la union

In [1]:
import pandas as pd
import collections

from docxtpl import DocxTemplate

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V3 - 2023-11-08.xlsx'

In [5]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE POR GRUPO')

C:\Users\Usuario\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Usuario\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [7]:
xlsx_entregas.head(3)

,INST,GRUPO,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,PAQ,TM. ENTREGADAS,AVANCE,INICIO DE ENTREGAS,ULTIMA ENTREGA,COD_COSECHA
0,1,1,2710,DEMIQUEL JIMENEZ JOSELITO,1054.46,NaN,1482.15,1.405601,NaN,NaN,287.0
1,1,1,2751,DEMIQUEL TERESA JIMENEZ DE,114.10,NaN,120.89,1.059509,NaN,NaN,287.0
2,1,1,7572,LAVERAN VACA ZENON,64.75,NaN,42.35,0.654054,NaN,NaN,286.0


In [8]:
xlsx_avance.head(3)

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,...,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,1,26449.77,32068.61,1.212434,543.38,543.38,NaN,1.000000,59.016913,0.212434,...,30115.348382,1953.261618,0.939091,NaN,-5618.84,5618.840000,NaN,NaN,NaN,NaN
1,2,10848.49,12296.90,1.133513,170.47,170.47,NaN,1.000000,72.135273,0.133513,...,8677.777218,3619.122782,0.705688,NaN,-1448.41,1448.410000,NaN,NaN,NaN,NaN
2,3,5390.78,6233.97,1.156413,100.01,94.56,5.45,0.945505,65.926079,0.210908,...,4517.391536,1716.578464,0.724641,271.425777,-843.19,1114.615777,NaN,NaN,NaN,NaN


## Argumentos

In [9]:
fecha_inf = '10 de noviembre de 2023'
fecha_form = '10-11-2023'

fecha_img = '10 de noviembre de 2023'
cod_grupos = [510, 277, 432, 158, 310, 98, 324, 323, 437, 115, 156, 205]

In [10]:
for i in cod_grupos:
    # datos del grupo
    grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == i].copy()
    grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
    grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
    grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
    
    # convertir datos del grupo a objetos
    list_obj_caneros = []
    for index, row in grupo.iterrows():
        canero = {
            'inst' : row['INST'],
            'cod_ca' : row['CODIGO CAÑERO '],
            'nom_ca' : row['NOMBRE CAÑERO'],
            'compromiso' : row['COMPROMISO DE ENTREGA'],
            'entrega' : row['TM. ENTREGADAS'],
            'entrega_porcen' : row['avance%']
        }
        list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
    
    # totales de los datos del grupo
    grupo_totales = {
        'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
        'entregas' : grupo['TM. ENTREGADAS'].sum(),
        'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
    }
    obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
    
    # datos generales de avance de cosecha
    avance = xlsx_avance[xlsx_avance['COD_COS'] == i]
    avance_totales = {
        'fecha_img' : fecha_img,
        'fecha_informe' : fecha_inf,
        'compromiso' : float(avance['COMPROMISO']),
        'entrega' : float(avance['ENTREGA']),
        'entrega_porcen' : float(avance['_ENTREGA']) * 100,
        'ha_canha' : float(avance['TOTAL_AREA']),
        'ha_cosecha' : float(avance['COSECHA']),
        'cosecha_porcen' : float(avance['%COSECHA']) * 100,
        'tch' : float(avance['TCH'])
    }
    
    # datos generales a objeto
    obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
    
    # generar reporte
    doc = DocxTemplate(ruta + '/templates/tpl_rpt_avance_posible_desvio.docx')
    context = {
        'insp' : list_obj_caneros,
        'grupo_totales' : obj_grupo_totales,
        'avance' : obj_avance_totales
    }
    doc.render(context)
    doc.save(ruta + '/informes/_' + str(grupo['CODIGO CAÑERO '].iloc[0]) + '_RDC_' + fecha_form + '_' + grupo['NOMBRE CAÑERO'].iloc[0] + '.docx')
    print('Se generó reporte de grupo:', i)

Se generó reporte de grupo: 510
Se generó reporte de grupo: 277
Se generó reporte de grupo: 432
Se generó reporte de grupo: 158
Se generó reporte de grupo: 310
Se generó reporte de grupo: 98
Se generó reporte de grupo: 324
Se generó reporte de grupo: 323
Se generó reporte de grupo: 437
Se generó reporte de grupo: 115
Se generó reporte de grupo: 156
Se generó reporte de grupo: 205


## DATOS DEL GRUPO

In [32]:
cod_grupo = 425

In [33]:
grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == cod_grupo].copy()
grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
grupo

,INST,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,TM. ENTREGADAS,avance%
1226,138,7542,LAZARTE PEREZ VICTOR,15137.2,7872.39,52.00691


In [43]:
grupo['CODIGO CAÑERO '].iloc[0]

10295

In [34]:
list_obj_caneros = []
for index, row in grupo.iterrows():
    canero = {
        'inst' : row['INST'],
        'cod_ca' : row['CODIGO CAÑERO '],
        'nom_ca' : row['NOMBRE CAÑERO'],
        'compromiso' : row['COMPROMISO DE ENTREGA'],
        'entrega' : row['TM. ENTREGADAS'],
        'entrega_porcen' : row['avance%']
    }
    list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_caneros

[canero(inst=138, cod_ca=7542, nom_ca='LAZARTE PEREZ VICTOR', compromiso=15137.2, entrega=7872.39, entrega_porcen=52.00691012868959)]

In [35]:
grupo_totales = {
    'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
    'entregas' : grupo['TM. ENTREGADAS'].sum(),
    'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
}
obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

grupo_totales(compromiso=15137.2, entregas=7872.39, entrega_porcen=52.00691012868959)

## DATOS DE AVANCE

In [36]:
xlsx_avance.head(3)

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,...,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,1,26449.77,28938.76,1.094103,543.39,463.66,79.73,0.853273,62.413751,0.240830,...,26791.513670,2147.246330,0.925800,3323.834712,-2488.99,5812.824712,NaN,NaN,NaN,NaN
1,2,10848.49,12015.99,1.107619,170.47,165.21,5.26,0.969144,72.731614,0.138475,...,8421.849103,3594.140897,0.700887,255.928114,-1167.50,1423.428114,NaN,NaN,NaN,NaN
2,3,5390.78,5359.97,0.994285,100.02,76.67,23.35,0.766547,69.909613,0.227738,...,3797.599057,1562.370943,0.708511,991.218256,30.81,960.408256,NaN,NaN,NaN,NaN


In [15]:
avance = xlsx_avance[xlsx_avance['COD_COS'] == cod_grupo]
avance

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA
28,30,65493.32,42091.86,0.642689,856.71,452.11,404.6,0.527728,93.100927,0.114961,857.018376,76.385829,65464.059007,34547.228021,7544.631979,0.820758,30916.830986,23401.46,7515.370986


In [29]:
avance_totales = {
    'fecha_img' : fecha_img,
    'fecha_informe' : fecha_inf,
    'compromiso' : float(avance['COMPROMISO']),
    'entrega' : float(avance['ENTREGA']),
    'entrega_porcen' : float(avance['_ENTREGA']) * 100,
    'ha_canha' : float(avance['TOTAL_AREA']),
    'ha_cosecha' : float(avance['COSECHA']),
    'cosecha_porcen' : float(avance['%COSECHA']) * 100,
    'tch' : float(avance['TCH'])
}
avance_totales

{'fecha_img': '05 de agosto de 2023',
 'fecha_informe': '14 de agosto de 2023',
 'compromiso': 65493.32,
 'entrega': 42091.86,
 'entrega_porcen': 64.26893612966941,
 'ha_canha': 856.71,
 'ha_cosecha': 452.11,
 'cosecha_porcen': 52.77281693922097,
 'tch': 93.10092676561014}

In [67]:
obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
obj_avance_totales

avance_totales(fecha_img='05 de agosto de 2023', fecha_informe='14 de agosto de 2023', compromiso=26449.77, entrega=19161.65, entrega_porcen=72.44543147256101, ha_canha=543.38, ha_cosecha=304.38, cosecha_porcen=56.01604770142442, tch=62.953052105920236)

In [68]:
doc = DocxTemplate(ruta + '/templates/tbl_rpt_posible_desvio.docx')
context = {
    'insp' : list_obj_caneros,
    'grupo_totales' : obj_grupo_totales,
    'avance' : obj_avance_totales
}
doc.render(context)
doc.save(ruta + '/informes/rpt.docx')